In [229]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import auth
import pandas as pd

In [230]:
scope = "playlist-modify user-read-private user-read-recently-played user-library-read user-read-currently-playing"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=auth.CLIENT_ID, client_secret=auth.CLIENT_SECRET, redirect_uri=auth.REDIRECT_URI))

In [231]:
def get_all_current_user_saved_tracks():
    offset = 0
    saved_tracks_group = []

    while(True):
        
        saved_tracks = sp.current_user_saved_tracks(limit=50, offset=offset)
        
        if len(saved_tracks['items']) == 0:
            break
        
        artist_names = []
        artist_ids = []
        track_names = []
        track_ids = []

        for item in saved_tracks["items"]:
            artist_name = item["track"]["artists"][0]["name"]
            artist_id = item["track"]["artists"][0]["id"]
            track_name = item["track"]["name"]
            track_id = item["track"]["id"]

            artist_names.append(artist_name)
            artist_ids.append(artist_id)
            track_names.append(track_name)
            track_ids.append(track_id)

        saved_tracks_group.append(pd.DataFrame({
            "Artist Name": artist_names,
            "Artist ID": artist_ids,
            "Track Name": track_names,
            "Track ID": track_ids
        }))
        offset += 50

    saved_tracks_df = pd.concat(saved_tracks_group)

    return saved_tracks_df

In [232]:

def get_current_user_recently_played():
    recently_played = sp.current_user_recently_played(limit=50)

    artist_names = []
    artist_ids = []
    track_names = []
    track_ids = []

    for item in recently_played["items"]:
        artist_name = item["track"]["artists"][0]["name"]
        artist_id = item["track"]["artists"][0]["id"]
        track_name = item["track"]["name"]
        track_id = item["track"]["id"]
        
        artist_names.append(artist_name)
        artist_ids.append(artist_id)
        track_names.append(track_name)
        track_ids.append(track_id)

    recently_played_df = pd.DataFrame({
        "Artist Name": artist_names,
        "Artist ID": artist_ids,
        "Track Name": track_names,
        "Track ID": track_ids
    })

    return recently_played_df

In [233]:
def get_recommendation_genre_seeds():
    genres_dict = sp.recommendation_genre_seeds()
    genres_df = pd.DataFrame.from_dict(genres_dict)

    return genres_df

In [234]:
def get_currently_playing():
    currently_playing = sp.current_user_playing_track()

    if(currently_playing['context']):
        context_type = currently_playing['context']['type']
        uri = currently_playing['context']['uri']
    else:
        context_type = currently_playing['context']
        uri = currently_playing['context']

    track_name = currently_playing['item']['name']
    track_id = currently_playing['item']['id']
    album_name = currently_playing['item']['album']['name']
    album_id = currently_playing['item']['album']['id']

    artist_names = []
    artist_ids = []

    for artist in currently_playing['item']['artists']:
        artist_name = artist['name']
        artist_id = artist['id']
        
        artist_names.append(artist_name)
        artist_ids.append(artist_id)

    currently_playing_df = pd.DataFrame({
        'Context Type': context_type,
        'URI': uri,
        'Track Name': track_name,
        'Track ID': track_id,
        'Album Name': album_name,
        'Album ID': album_id,
        'Artist Names': [artist_names],
        'Artist IDs': [artist_ids]
        })

    return currently_playing_df

In [235]:
import numpy as np
from multimethod import multimethod

In [236]:
@multimethod
def get_track_features(tracks: list):
    audio_features = sp.audio_features(tracks)
    
    tracks_details = []
    
    for item in tracks:
        tracks_details.append(sp.track(item))

    track_id = []
    track_name = []
    popularity = []
    duration_ms = []
    explicit = []
    artists_name = []
    artists_id = []
    release_date = []
    danceability = []
    energy = []
    key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    time_signature = []
    
    # uri
    # track_href
    # analysis_url

    for audio in zip(audio_features, tracks_details):
        track_id.append(audio[0]['id'])
        track_name.append(audio[1]['name'])
        popularity.append(audio[1]['popularity'])
        duration_ms.append(audio[0]['duration_ms'])
        explicit.append(audio[1]['explicit'])
        release_date.append(audio[1]['album']['release_date'])
        danceability.append(audio[0]['danceability'])
        energy.append(audio[0]['energy'])
        key.append(audio[0]['key'])
        loudness.append(audio[0]['loudness'])
        mode.append(audio[0]['mode'])
        speechiness.append(audio[0]['speechiness'])
        acousticness.append(audio[0]['acousticness'])
        instrumentalness.append(audio[0]['instrumentalness'])
        liveness.append(audio[0]['liveness'])
        valence.append(audio[0]['valence'])
        tempo.append(audio[0]['tempo'])
        time_signature.append(audio[0]['time_signature'])
        # artist_name = []
        # artist_id = []
        # for artist in audio[1]['album']['artists']:
        #     artist_name.append(artist['name'])
        #     artist_id.append(artist['id'])
        # artists_name.append(artist_name)
        # artists_id.append(artist_id)
        artists_name.append([artist['name'] for artist in audio[1]['album']['artists']])
        artists_id.append([artist['id'] for artist in audio[1]['album']['artists']])

    track_features_df = pd.DataFrame({
        'track_id': track_id,
        'track_name': track_name,
        'popularity': popularity,
        'duration_ms': duration_ms,
        'explicit': explicit,
        'artists_name': np.array(artists_name, dtype=object),
        'artists_id': np.array(artists_id, dtype=object),
        # 'artists_name': [artists_name],
        # 'artists_id': [artists_id],
        'release_date': release_date,
        'danceability': danceability,
        'energy': energy,
        'key': key,
        'loudness': loudness,
        'mode': mode,
        'speechiness': speechiness,
        'acousticness': acousticness,
        'instrumentalness': instrumentalness,
        'liveness': liveness,
        'valence': valence,
        'tempo': tempo,
        'time_signature': time_signature
    })

    return track_features_df

In [237]:
@multimethod
def get_track_features(track: str):
    audio_features = sp.audio_features(track)[0]
    
    tracks_details = sp.track(track)

    track_id = audio_features['id']
    track_name = tracks_details['name']
    popularity = tracks_details['popularity']
    duration_ms = audio_features['duration_ms']
    explicit = tracks_details['explicit']
    release_date = tracks_details['album']['release_date']
    danceability = audio_features['danceability']
    energy = audio_features['energy']
    key = audio_features['key']
    loudness = audio_features['loudness']
    mode = audio_features['mode']
    speechiness = audio_features['speechiness']
    acousticness = audio_features['acousticness']
    instrumentalness = audio_features['instrumentalness']
    liveness = audio_features['liveness']
    valence = audio_features['valence']
    tempo = audio_features['tempo']
    time_signature = audio_features['time_signature']
    artists_name = [artist['name'] for artist in tracks_details['album']['artists']]
    artists_id = [artist['id'] for artist in tracks_details['album']['artists']]

    track_features_df = pd.DataFrame({
        'track_id': track_id,
        'track_name': track_name,
        'popularity': popularity,
        'duration_ms': duration_ms,
        'explicit': explicit,
        'artists_name': [artists_name],
        'artists_id': [artists_id],
        'release_date': release_date,
        'danceability': danceability,
        'energy': energy,
        'key': key,
        'loudness': loudness,
        'mode': mode,
        'speechiness': speechiness,
        'acousticness': acousticness,
        'instrumentalness': instrumentalness,
        'liveness': liveness,
        'valence': valence,
        'tempo': tempo,
        'time_signature': time_signature
    })

    return track_features_df

In [238]:
import re

In [258]:
from unidecode  import unidecode 

In [259]:
def get_track_id(origin_track: str, origin_artist, limit=10, offset=0, type='track', market=None):
    
    UNVALID_CHARS = r"[\"'/,.:%?!;()-É]"

    track = re.sub(UNVALID_CHARS, "", origin_track)
    # track = track.replace(' ', '%20')
    track = unidecode(track)


    artist = ' '.join(origin_artist)
    artist = unidecode(artist)
    # artist = re.sub(UNVALID_CHARS, "", artist)
    artist = artist.replace(' ', '%20')
    
    q = f'{track}%20track:{track}%20artist:{artist}'

    try:
        search_result = sp.search(q, limit=1, offset=0, type='track', market=None)
    except:
        return origin_track

    try:
        return search_result['tracks']['items'][0]['id']
    except:
        return origin_track